Imports

In [1]:
# Import necessary libraries
from nnsight import LanguageModel
import torch
import pandas as pd

# Initialize the language model with specific parameters
model = LanguageModel("google/gemma-2b-it", trust_remote_code=True, device_map="cuda:1", low_cpu_mem_usage=True, torch_dtype=torch.float16)
model.requires_grad_(False)



from sae_lens import SAE
layer = 6

# get the SAE for this layer
sae, cfg_dict, _ = SAE.from_pretrained(
    release = "gemma-2b-res-jb",
    sae_id = f"blocks.{layer}.hook_resid_post",
    device = 'cuda:1'
)



Parameters

In [99]:
#@title Self-explanation in 18 lines
#feature = 471  #@param {type: "integer"}
#feature = 5892  #@param {type: "integer"}
# feature = 9415
feature = 416    
scale = 25.5 #@param {type: "number"}
se_demo = True  #@param {type: "boolean"}
max_new_tokens = 80  #@param {type: "integer"}
n_generate = 1  #@param {type: "integer"}
vals = ['positive', 'neutral', 'negative']
samples = pd.read_excel('/home/ashater/work/sae/sentence_sentiment_gpt4o.xlsx')

In [43]:
df0_5000 = pd.read_csv('/home/ashater/work/sae/0_500.csv', index_col=0)
df0_5000 = df0_5000.map(lambda x: eval(x))
df0_5000['scale'] = df0_5000['0'].map(lambda x: x[0])
df0_5000['explain'] = df0_5000['0'].map(lambda x: x[1])

df5000 = pd.read_csv('/home/ashater/work/sae/500_max.csv', index_col=0)
df5000 = df5000.map(lambda x: eval(x))
df5000['scale'] = df5000['0'].map(lambda x: x[0])
df5000['explain'] = df5000['0'].map(lambda x: x[1])
df5000.index += 5000


In [79]:

#pd.concat([df0_5000, df5000], axis=0).to_excel('/home/ashater/work/sae/sae_features_scale.xlsx')
sae_explain = pd.concat([df0_5000, df5000], axis=0)


sae_explain.loc[5347]
# sae_explain.loc[[11912,  3390, 15881,  5347, 16334,   556, 11785,  1630,  5490,  1587,
#           5624,  5396,  1491, 12388,  6877, 10493, 13816, 14183,  8350,  3109]]

0          (41.111111111111114, ['something that is the o...
scale                                              41.111111
explain    ['something that is the opposite of." For exam...
Name: 5347, dtype: object

## Main code prompt


In [92]:


v = ["How does does financial sentiment affect credit rating of , particular related to stock price?", "what is IBM company credit rating and how does  it effect lending?", "Stock rallied on good news"]
topics = [  416,   802,  1520,  1809,  2285,  2726,  2743,  3102,  3394,  3961,
        4731,  4889,  5280,  5463,  5938,  6499,  7281,  7396,  7437,  7817,
        7999,  8038,  8165,  8265,  9464,  9570,  9628,  9731, 10151, 10349,
       10454, 10637, 11607, 11882, 12893, 14039, 15746, 16271]

In [106]:
full_res = []
#v = range(0,10)
for ismpl in v:


    prompt = """<start_of_turn>user\nCan you rate  the following sentnece as a financial credit sentiment, very positive, 
    somewhat positive, neutral, somewhat negative, 
    very negative? """ + ismpl + '<end_of_turn>\n<start_of_turn>model\nThe sentiment of the setence is "X"'

    #prompt = '<start_of_turn>user\nWhat is the meaning of the word "X"?<end_of_turn>\n<start_of_turn>model\nThe meaning of the word "X" is "'

    # prompt = '<start_of_turn>user\n' + ismpl +  '<end_of_turn>\n<start_of_turn>model X'
    positions = [i for i, a in enumerate(model.tokenizer.encode(prompt)) if model.tokenizer.decode([a]) == "X"]
    with model.generate(prompt, max_new_tokens=max_new_tokens, num_return_sequences=n_generate, do_sample=True, scan=False, validate=False) as gen:
        # vector steering section
        # for feature_i in topics:
        vector = sae.W_dec[[feature]]
        vector = vector / vector.norm()
        vector = vector * scale
        # setting vector into layer 2
        # for position in positions:
        #   model.model.layers[2].output[0][:, position] = vector
            
        # get final output saved
        out = model.generator.output.save()
        # capture at layer 6 or 12
        resid = model.model.layers[12].output.save()
        
    features = sae.encode(resid[0])
    summed_activations = features.abs().sum(dim=1) # Sort by max activations
    top_activations_indices = summed_activations.topk(20).indices # Get indices of top 20    

    for i, l in enumerate(model.tokenizer.batch_decode(out)):
        s = repr(l.partition(prompt)[2].partition("<eos>")[0])
        print(prompt + "|"+ s + "|" + str(top_activations_indices))
        # print(str(top_activations_indices))


<start_of_turn>user
Can you rate  the following sentnece as a financial credit sentiment, very positive, 
    somewhat positive, neutral, somewhat negative, 
    very negative? How does does financial sentiment affect credit rating of , particular related to stock price?<end_of_turn>
<start_of_turn>model
The sentiment of the setence is "X"|" which is classified as somewhat negative. \n\nFinancial sentiment, especially when it is negative, can influence, and is often guided by, an individual's credit rating. A negative sentiment can raise concerns about an individual's ability to repay debts, and thus, may negatively impact their credit rating. As a result, a negative sentiment can lead to higher interest rates or loan costs for that individual."|tensor([[11912, 10859, 15760, 15202,  3390, 15881, 13327,  5347,  8814, 16334,
           556, 11785,  8704,  5624, 12388, 11160,  9415,  5490,  1630,  3586]],
       device='cuda:1')
<start_of_turn>user
Can you rate  the following sentnece as 

## summed activations across tokens

In [30]:
summed_activations.shape

torch.Size([1, 16384])

## Get features for each token

In [36]:
features.shape

torch.Size([1, 21, 16384])